In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krish'\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd

# Load dataset (after you download from Kaggle)
df = pd.read_csv(r'Q:\cyberbulyying\cyberbullying_tweets.csv')
print(df.head())

                                          tweet_text cyberbullying_type
0  In other words #katandandre, your food was cra...  not_cyberbullying
1  Why is #aussietv so white? #MKR #theblock #ImA...  not_cyberbullying
2  @XochitlSuckkks a classy whore? Or more red ve...  not_cyberbullying
3  @Jason_Gio meh. :P  thanks for the heads up, b...  not_cyberbullying
4  @RudhoeEnglish This is an ISIS account pretend...  not_cyberbullying


In [4]:
stop = set(stopwords.words('english'))

def clean_text(text):
    text = str(text)
    text = re.sub(r"http\S+|www.\S+", "", text)   # remove URLs
    text = re.sub(r"@\w+|#\w+", "", text)         # remove @user and hashtags
    text = re.sub(r"[^a-zA-Z\s]", "", text)       # keep only letters
    text = text.lower()
    text = ' '.join([word for word in text.split() if word not in stop])
    return text

df['clean_text'] = df['tweet_text'].apply(clean_text)
print(df[['tweet_text','clean_text']].head())


                                          tweet_text  \
0  In other words #katandandre, your food was cra...   
1  Why is #aussietv so white? #MKR #theblock #ImA...   
2  @XochitlSuckkks a classy whore? Or more red ve...   
3  @Jason_Gio meh. :P  thanks for the heads up, b...   
4  @RudhoeEnglish This is an ISIS account pretend...   

                                          clean_text  
0                            words food crapilicious  
1                                              white  
2                   classy whore red velvet cupcakes  
3  meh p thanks heads concerned another angry dud...  
4  isis account pretending kurdish account like i...  


In [5]:
embeddings_index = {}
with open("Q:\\glove.6B\\glove.6B.300d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype="float32")
        embeddings_index[word] = coefs
print("Loaded %s word vectors." % len(embeddings_index))


Loaded 400000 word vectors.


In [6]:
def get_glove_vector(text):
    words = text.split()
    if len(words) == 0:
        return np.zeros(300)
    vectors = [embeddings_index.get(w, np.zeros(300)) for w in words]
    return np.mean(vectors, axis=0)

X_glove = np.vstack(df['clean_text'].apply(get_glove_vector))
print("Feature matrix shape:", X_glove.shape)


Feature matrix shape: (47692, 300)


In [7]:
from sklearn.decomposition import PCA
pca = PCA(n_components=100)   # paper reduced to 9000; smaller for demo
X_pca = pca.fit_transform(X_glove)
print("Reduced feature matrix:", X_pca.shape)


Reduced feature matrix: (47692, 100)


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(df['cyberbullying_type'])   # column name may vary

X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y)


In [16]:
# Prepare text-based train/test splits for the Hugging Face dataset (keeps indices aligned)
from sklearn.model_selection import train_test_split
# Ensure 'clean_text' exists; if not, create it from 'tweet_text'
if 'clean_text' not in df.columns:
    if 'tweet_text' in df.columns:
        import re
        from nltk.corpus import stopwords
        stop = set(stopwords.words('english'))
        def _clean_text(t):
            t = str(t)
            t = re.sub(r"http\S+|www.\S+", "", t)
            t = re.sub(r"@\w+|#\w+", "", t)
            t = re.sub(r"[^a-zA-Z\s]", "", t)
            t = t.lower()
            t = ' '.join([w for w in t.split() if w not in stop])
            return t
        df['clean_text'] = df['tweet_text'].apply(_clean_text)
    else:
        raise KeyError("Neither 'clean_text' nor 'tweet_text' found in df.")

# texts list aligned with labels in `y`
texts = df['clean_text'].tolist()
# If `y` was not created earlier, compute it here
if 'y' not in globals():
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(df['cyberbullying_type'])

# Split texts and labels together so indices remain aligned
X_train_texts, X_test_texts, y_train, y_test = train_test_split(texts, y, test_size=0.2, random_state=42, stratify=y)

# Create TweetDataset instances (requires TweetDataset class and tokenizer to be defined)
# If the TweetDataset class hasn't been defined (cell order/execution), define a compatible fallback here.
if 'TweetDataset' not in globals():
    # import torch only if not already available to avoid duplicate imports
    if 'torch' not in globals():
        import torch
    from torch.utils.data import Dataset

    class TweetDataset(Dataset):
        def __init__(self, texts, labels, tokenizer, max_len=64):
            self.texts = texts
            self.labels = labels
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])
            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_tensors='pt'
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'labels': torch.tensor(self.labels[idx], dtype=torch.long)
            }

if 'tokenizer' not in globals():
    from transformers import RobertaTokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

train_ds = TweetDataset(X_train_texts, y_train, tokenizer)
test_ds = TweetDataset(X_test_texts, y_test, tokenizer)
print('Created train_ds and test_ds with sizes:', len(train_ds), len(test_ds))
print('Example item from train_ds:')
print(train_ds[0])


Created train_ds and test_ds with sizes: 38153 9539
Example item from train_ds:
{'input_ids': tensor([    0,   506, 24029,  1099,  1114,   224,  6587,  5738,  4181,   192,
            2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'labels': tensor(4)}


In [ ]:
%pip uninstall -y keras tensorflow


Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install tensorflow==2.14 keras==2.14


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.14 (from versions: 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.14


In [ ]:
%pip install torch torchvision torchaudio transformers --upgrade


Note: you may need to restart the kernel to use updated packages.


In [19]:
# Install accelerate (required by Trainer) and torch extras for transformers; run in the notebook kernel.
%pip install "accelerate>=0.26.0" --quiet
%pip install "transformers[torch]" --quiet
print('Installed accelerate and transformers[torch] (if not already). You may need to restart the kernel after installation.')


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Installed accelerate and transformers[torch] (if not already). You may need to restart the kernel after installation.
Note: you may need to restart the kernel to use updated packages.
Installed accelerate and transformers[torch] (if not already). You may need to restart the kernel after installation.


In [22]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch
import inspect

class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=64):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# Prefer text-based datasets created earlier (train_ds/test_ds). If they don't exist, create from text splits.
if 'train_ds' not in globals():
    if 'X_train_texts' in globals():
        train_ds = TweetDataset(X_train_texts, y_train, tokenizer)
        test_ds = TweetDataset(X_test_texts, y_test, tokenizer)
    else:
        raise NameError('train_ds not found and text-based splits are missing. Run preprocessing cells first.')

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=len(np.unique(y)))
# Build TrainingArguments in a backward-compatible way
common_args = dict(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10,

)

# Check if TrainingArguments supports 'evaluation_strategy' (newer transformers)
if 'evaluation_strategy' in inspect.signature(TrainingArguments.__init__).parameters:
    training_args = TrainingArguments(
        **common_args,
        evaluation_strategy='epoch',
        save_strategy='epoch',
    )
else:
    # Fallback for older transformers: use eval_steps/save_steps instead (one per epoch)
    steps_per_epoch = max(1, len(train_ds) // common_args['per_device_train_batch_size'])
    training_args = TrainingArguments(
        **common_args,
        eval_steps=steps_per_epoch,
        save_steps=steps_per_epoch,
    )
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
)
trainer.train() 
print("Training complete.")# Cyberbullying Detection using GloVe and RoBERTa

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
metrics = trainer.evaluate()
print(metrics)


NameError: name 'trainer' is not defined

In [20]:
from lime.lime_text import LimeTextExplainer

explainer = LimeTextExplainer(class_names=le.classes_)

def predict_proba(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors='pt')
    outputs = model(**tokens)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1).detach().numpy()
    return probs

idx = 0
exp = explainer.explain_instance(df['clean_text'].iloc[idx], predict_proba, num_features=10)
exp.show_in_notebook(text=True)


ModuleNotFoundError: No module named 'lime'

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', probability=True, random_state=42),
    "NaiveBayes": GaussianNB(),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, average='macro')
    rec = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    results.append([name, acc, prec, rec, f1])

results_df = pd.DataFrame(results, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1'])
print(results_df)


In [ ]:
from sklearn.model_selection import KFold, cross_val_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_acc = cross_val_score(rf, X_pca, y, cv=kf, scoring='accuracy')
cv_prec = cross_val_score(rf, X_pca, y, cv=kf, scoring='precision_macro')
cv_rec = cross_val_score(rf, X_pca, y, cv=kf, scoring='recall_macro')
cv_f1 = cross_val_score(rf, X_pca, y, cv=kf, scoring='f1_macro')

print(f"Average Accuracy: {cv_acc.mean():.2f}")
print(f"Average Precision: {cv_prec.mean():.2f}")
print(f"Average Recall: {cv_rec.mean():.2f}")
print(f"Average F1 Score: {cv_f1.mean():.2f}")
